In [1]:
import pandas as pd
import pickle
import numpy as np 
import matplotlib.pyplot as plt 
import os,sys
os.chdir('../')
sys.path.append('./src/data')
sys.path.append('./src/probability')
sys.path.append('./src/events')
from DC.importer import MC2018_DC
from DC.processer import get_flux, interpolate_flux_DC
from numerical import P_num_over_E
from numerical import P_num
from functions import nufit_params_nsi
from numerical import wrapper

In [2]:
mc = MC2018_DC()
flux_df = interpolate_flux_DC()

In [3]:
numu = mc['pdg'] == 14
mc[numu]

,pdg,pid,reco_coszen,reco_energy,true_coszen,true_energy,type,weight
135870,14.0,0.0,0.125,6.6,0.271757,4.535313,3.0,2.973662e-10
135871,14.0,0.0,-0.375,11.7,0.206704,3.246002,2.0,1.137619e-10
135872,14.0,0.0,-0.875,8.7,-0.748358,3.008176,1.0,9.160223e-11
135873,14.0,0.0,0.375,15.6,0.190249,4.417644,2.0,2.748287e-10
135874,14.0,0.0,-0.125,11.7,-0.002781,4.879795,2.0,3.665623e-10
...,...,...,...,...,...,...,...,...
760812,14.0,0.0,0.375,49.2,-0.489007,141.901138,0.0,8.110281e-06
760813,14.0,1.0,0.125,20.7,0.407026,93.600182,0.0,2.396959e-06
760814,14.0,1.0,0.625,36.9,0.078068,215.255844,0.0,2.796840e-05
760815,14.0,0.0,0.125,20.7,0.355241,84.537041,0.0,1.755870e-06


In [4]:
df = mc[numu].sample(2000)
#get_flux('m',df['true_energy'], df['true_coszen'], flux_df)

In [5]:
import time 
E = df['true_energy'].values
z = df['true_coszen'].values
'''
start = time.time()
r= [P_num_over_E('m', [E[i]],zenith=z[i], nsi=False) for i in range(len(E))]
print(time.time() - start)
'''

"\nstart = time.time()\nr= [P_num_over_E('m', [E[i]],zenith=z[i], nsi=False) for i in range(len(E))]\nprint(time.time() - start)\n"

In [ ]:
import time 
from multiprocessing import Pool
p = Pool()
start = time.time()
r= [P_num_over_E('m', df['true_energy'].values,zenith=z, nsi=False) for z in df['true_coszen'].values]
print(time.time() - start)

In [45]:

start = time.time()
r = np.array([wrapper(['m', [E[i]],z, False, nufit_params_nsi, 3, True])[1] for i,z in enumerate(z)])
print(time.time() - start)
#11-9 sec

10.062361001968384


In [7]:
from PINGU.processer import generate_probabilities_PINGU
start = time.time()
generate_probabilities_PINGU('m','m',E, z, 0,0,nufit_params_nsi,False,0,3,True)

9.297943115234375


In [103]:
param_list = [nufit_params_nsi, nufit_params_nsi]
arg_tuples = [(i,j, p) for i in range(2) for j in range(2) for p in param_list]
rng = np.random.default_rng(12345)
rng.shuffle(arg_tuples)
split_arg_tuples=  np.array_split(arg_tuples,2)[0]
split_arg_tuples[0]

array([1, 0,
       {'theta_12': 0.5836381018669037, 'theta_13': 0.14957471689591406, 'theta_23': 0.8587019919812102, 'theta_34': 0.0, 'theta_24': 0.2013579207903308, 'theta_14': 0.0, 'delta_ij': 3.4033920413889422, 'dm_21': 7.42e-05, 'dm_31': 0.002517, 'dm_41': 0.93, 'e_ee': 0, 'e_em': 0, 'e_et': 0, 'e_mm': 0, 'e_mt': 0, 'e_tt': 0, 'e_es': 0, 'e_ms': 0, 'e_ts': 0, 'e_ss': 0}],
      dtype=object)

In [111]:
100 % 100 == 0

True